In [12]:
import json
import os
import sklearn.metrics
import numpy as np
from pprint import pprint

In [1]:
def evaluate(tasks, berts, domains, runs=10, testing=None):
    print("running...")
    for task in tasks:
        for bert in berts:
            for domain in domains:            
                for new_run in range(1, 10):
                    scores=[]
                    for run in range(1, runs+1):
                        DATA_DIR=os.path.join(task, domain)
                        print(DATA_DIR)
                        
                        if testing:
                            OUTPUT_DIR=os.path.join("run", bert+"_"+task, domain, str(run))
                            print(OUTPUT_DIR)
                        else:
                            OUTPUT_DIR=os.path.join("run", bert+"_"+task, domain, str(new_run), str(run))
                            print(OUTPUT_DIR)
                        # if os.path.exists(os.path.join(OUTPUT_DIR, "predictions.json") ):
                        if "rrc" in task:
                            ret=!python eval/evaluate-v1.1.py $DATA_DIR/test.json $OUTPUT_DIR/predictions.json
                            score=json.loads(ret[0])
                            scores.append([score["exact_match"], score["f1"] ] )
                        elif "ae" in task:
                            ret=!python eval/evaluate_ae.py --pred_json $OUTPUT_DIR/predictions.json
                            # print(scores)
                            scores.append(float(ret[0])*100 )
                        elif "asc" in task:
                            with open(os.path.join(OUTPUT_DIR, "predictions.json") ) as f:
                                results=json.load(f)
                            y_true=results['label_ids']
                            y_pred=[np.argmax(logit) for logit in results['logits'] ]
                            p_macro, r_macro, f_macro, _=sklearn.metrics.precision_recall_fscore_support(y_true, y_pred, average='macro')
                            f_macro = 2*p_macro*r_macro/(p_macro+r_macro)
                            scores.append([100*sklearn.metrics.accuracy_score(y_true, y_pred), 100*f_macro ] )
                        else:
                            raise Exception("unknown task")
                    scores=np.array(scores)
                    m=scores.mean(axis=0)

                    if len(scores.shape)>1:
                        for iz, score in enumerate(m):
                            print(task, ":", bert, domain, "metric", iz, round(score, 2))
                            pprint(scores)
                            if testing:
                                np.savetxt(task + '_' + domain + '_ftb_test.csv', scores, delimiter=',')
                            else:
                                np.savetxt(str(new_run) + '_' + task + '_' + domain + '_ftb_valid.csv', scores, delimiter=',')
                    else:
                        np.savetxt(task + '_' + domain + '_ftb_test.csv', scores, delimiter=',')
                        print(task, ":", bert, domain, round(m,2))
                        pprint(scores)
                        break


In [29]:
evaluate(tasks, berts, domains, runs, testing)

running...
Using datadir of ae/laptop
Using outputdir of run/pt_ae/laptop/1/predictions.json
ae
running run/pt_ae/laptop/1
got ['Traceback (most recent call last):', '  File "../eval/evaluate_ae.py", line 174, in <module>', '    print(evaluate(args.pred_json, command, template) )', '  File "../eval/evaluate_ae.py", line 157, in evaluate', '    label_laptop_xml(template, command[4], pred_json["raw_X"], y_pred)', '  File "../eval/evaluate_ae.py", line 74, in label_laptop_xml', '    dom=ET.parse(fn)', '  File "/opt/conda/lib/python3.7/xml/etree/ElementTree.py", line 1197, in parse', '    tree.parse(source, parser)', '  File "/opt/conda/lib/python3.7/xml/etree/ElementTree.py", line 587, in parse', '    source = open(source, "rb")', "FileNotFoundError: [Errno 2] No such file or directory: 'ae/official_data/Laptops_Test_Data_PhaseA.xml'"]


ValueError: could not convert string to float: 'Traceback (most recent call last):'